# MMPBSA Version 1.2
ToDo:
- reading atom index from index.ndx
- optimize the calculation speed
- Entropy contribution
- Debye Hukel correclation
- use different atom radius
- optimize the workflow
- in parallel

LOG:
- use index.ndx to  designate groups
- support APBS to do pbsa 2023/03/17
- the basic calculation of MM energy is done
- use mda to read traj
- use radbondi from amber instead of calculated radius
- residue energy decompose 2023/3/12
- the calculation speed is optimized 8s->0.1s
- tricks: residue and Ligand energy is 50/50
- every frame MM energy data was output to xlsx file

In [70]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import math
import MDAnalysis as mda
import os

In [71]:
dt=10000  # ps

ligidx=2
proidx=1

tproutpath="1ebz/out.out"
indexpath="1ebz/index.ndx"
xtcpath="1ebz/traj.xtc"

# elec
MGset="mg-auto"
temp=298.15
pdie=2         # 溶质介电常数
sdie=78.54     # 溶剂介电常数, 真空1, 水78.54
pbm="npbe"     # PB方程求解方法, lpbe(线性), npbe(非线性), smbpe(大小修正)
bcfl="mdh"     # 粗略格点PB方程的边界条件, zero, sdh/mdh(single/multiple Debye-Huckel), focus, map       
srfm="smol"    # 构建介质和离子边界的模型, mol(分子表面), smol(平滑分子表面), spl2/4(三次样条/7阶多项式)   
chgm="spl4"    # 电荷映射到格点的方法, spl0/2/4, 三线性插值, 立方/四次B样条离散   
swin=0.3       # 立方样条的窗口值, 仅用于 srfm=spl2/4   
srad=1.4       # 溶剂探测半径  
sdens=10       # 表面密度, 每A^2的格点数, (srad=0)或(srfm=spl2/4)时不使用  
pcharge=1
pconc=0.15
pradius=0.95
ncharge=-1
nconc=0.15
nradius=1.81

# sasa
atemp=298.15
asrfm="sacc"
aswin=0.3
asrad=1.4
agamma=1
apress=0
abconc=0
asdens=10
adpos=0.2
agridx=0.1
agridy=0.1
agridz=0.1


gridType=1			# 格点大小 grid (0:GMXPBSA 1:psize)

cfac=3				# 分子尺寸到粗略格点的放大系数
					# Factor to expand mol-dim to get coarse grid dim
fadd=10				# 分子尺寸到细密格点的增加值(A)
					# Amount added to mol-dim to get fine grid dim (A)
df=0.5				# 细密格点间距(A) The desired fine mesh spacing (A)

gamma=0.02267788
offset=3.84928

In [72]:
with open(tproutpath) as f:
    lines=f.readlines()


with open(indexpath) as idxf:
    idxflines=idxf.readlines()
idxlist=[]
for idxfline in idxflines:
    if re.findall(re.compile(r"\[([\s0-9a-zA-Z]+)\]"),idxfline) !=[]:
        idxlist.append(idxflines.index(idxfline))

liglist=np.array(0)
prolist=np.array(0)
comlist=np.array(0)

if ligidx+1<len(idxlist):
    for i in range(idxlist[ligidx]+1,idxlist[ligidx+1]):
        liglist=np.insert(liglist,-1,idxflines[i].split())

elif ligidx+1==len(idxlist):
    for i in range(idxlist[ligidx]+1,len(idxflines)):
        liglist=np.insert(liglist,-1,idxflines[i].split())
else:
    print('ligand index in index.ndx error')

liglist=np.delete(liglist,-1)-1
liglist.sort()
    
if proidx+1<len(idxlist):
    for i in range(idxlist[proidx]+1,idxlist[proidx+1]):
        prolist=np.insert(prolist,-1,idxflines[i].split())

elif proidx+1==len(idxlist):
    for i in range(idxlist[proidx]+1,len(idxflines)):
        prolist=np.insert(prolist,-1,idxflines[i].split())
else:
    print('ligand index in index.ndx error')

prolist=np.delete(prolist,-1)-1
prolist.sort()

comlist=np.insert(prolist,-1,liglist)
comlist.sort()


In [73]:
# get all atoms ==gro
# 1. find all molblocks
molblockstart=[]
box=[]
for line in lines:
    if line.startswith("   molblock"):
        molblockstart.append(lines.index(line))
    if line.startswith("box (3x3):"):
        box.append(re.findall(re.compile(r"=\{([\,\s\+\-e\.0-9]+)\}"),lines[lines.index(line)+1])[0].split(","))
        box.append(re.findall(re.compile(r"=\{([\,\s\+\-e\.0-9]+)\}"),lines[lines.index(line)+2])[0].split(","))
        box.append(re.findall(re.compile(r"=\{([\,\s\+\-e\.0-9]+)\}"),lines[lines.index(line)+3])[0].split(","))
totalatom={}
for i,j in enumerate(molblockstart):
    totalatom[re.findall(re.compile(r"\s[0-9]+\s\"([0-9a-zA-Z\s\-\_\+]+)\""),lines[j+1])[0]]=re.findall(re.compile(r"\s=\s([0-9]+)"),lines[j+2])[0]

In [74]:
# 读取坐标
for line in lines:
    if line.startswith("x ("):
        xlen=re.findall(re.compile(r"\s\(([0-9]+)x"),line)
        xstart=lines.index(line)+1
        xend=int(xlen[0])+xstart
coord=[re.findall(re.compile(r"\{([\s0-9a-zA-Ze\.\+\-\,]+)\}"),line)[0].split(",") for line in lines[xstart:xend]]
coord=pd.DataFrame(coord,columns=["x","y","z"])

In [75]:
# 修改resindex思路，根据循环
# 先设置一个初始residx_tmp=0,在第一层循环时即对蛋白序列都只加上0
# 第二次循环时，residx_tmp取存储在tmp中的最后一个残基index进行累加
# 对于sol及NA离子，通过在循环内添加countindx实现自增
tmp=[]
type=re.compile(r"type=([\s0-9]+)")
mass=re.compile(r"m=([\s\.0-9e\+\-]+)")
charge=re.compile(r"q=([\s\.0-9e\+\-]+)")
adx=re.compile(r"atom\[([\s0-9]+)")
resid=re.compile(r"resind=([\s0-9\+\-]+)")
atn=re.compile(r"atomnumber=([\s0-9\+\-]+)")
atomname=re.compile(r"{name=\"([A-Za-z0-9\*\_]+)\"")
atomtype=re.compile(r"{name=\"([A-Za-z0-9\*\_]+)\",")
residuestart=[]
residx_tmp=0
for line in lines:
    for i,j in enumerate(totalatom):
        if line.startswith("      name="+"\""+j+"\""):
            Len=re.findall(re.compile(r"\s\(([0-9]+)"),lines[lines.index(line)+2])
            start=lines.index(line)+3
            end=start+int(Len[0])
            resn={}
            resns=re.findall(re.compile(r"\s\(([0-9]+)\)"),lines[start+3*int(Len[0])+2])[0]
            for res in range(int(resns)):
                resn[res]=re.findall(re.compile(r"{name=\"([a-zA-Z0-9\_\+\*]+)"),lines[start+3*int(Len[0])+res+3])[0]
            if int(totalatom[j])!=1:
                coutindx=1
                for n in range(int(totalatom[j])):
                    count=1
                    for m in lines[start:end]:
                        tmp.append([
                            re.findall(type,m)[0].split()[0],
                            re.findall(mass,m)[0].split()[0],
                            re.findall(charge,m)[0].split()[0],
                            re.findall(adx,m)[0].split()[0],
                            int(re.findall(resid,m)[0].split()[0])+coutindx+residx_tmp,
                            re.findall(atn,m)[0].split()[0],
                            re.findall(atomname,lines[start+int(Len[0])+count])[0].split()[0],
                            re.findall(atomtype,lines[start+2*int(Len[0])+count+1])[0].split()[0],
                            resn[int(re.findall(resid,m)[0].split()[0])],
                            j,
                        ])
                        count=count+1
                    coutindx=coutindx+1
                residx_tmp=tmp[-1][4]
            else:
                count=1
                for m in lines[start:end]:
                        tmp.append([
                            re.findall(type,m)[0].split()[0],
                            re.findall(mass,m)[0].split()[0],
                            re.findall(charge,m)[0].split()[0],
                            re.findall(adx,m)[0].split()[0],
                            int(re.findall(resid,m)[0].split()[0])+1+residx_tmp,
                            re.findall(atn,m)[0].split()[0],
                            re.findall(atomname,lines[start+int(Len[0])+count])[0].split()[0],
                            re.findall(atomtype,lines[start+2*int(Len[0])+count+1])[0].split()[0],
                            resn[int(re.findall(resid,m)[0].split()[0])],
                            j
                        ])
                        count=count+1
                residx_tmp=tmp[-1][4]
info=pd.DataFrame(tmp,columns=["functype","mass","charge","atomidx","resid","atomnumber","atomname","atomtype","resname","group"])

In [76]:
total=pd.concat([info,coord],axis=1)
total.insert(loc=0,column="index",value=np.arange(1,total.shape[0]+1))
total[["functype","atomidx","resid","atomnumber"]]=total[["functype","atomidx","resid","atomnumber"]].apply(pd.to_numeric)
total[["mass","charge","x","y","z"]]=total[["mass","charge","x","y","z"]].astype(float)

In [77]:
# extract C6 C12
for line in lines:
    if line.startswith("   ffparams:"):
        atnr=re.findall(re.compile(r"\satnr=([0-9]+)"),lines[lines.index(line)+1])
        atnr=int(atnr[0])*int(atnr[0])
        funcStart=lines.index(line)+3
        funcEnd=funcStart+atnr
func=lines[funcStart:funcEnd]
tmp=[]
# C6
for i in func:
    tmp.append([re.findall(re.compile(r"c6=\s([0-9\.\-e\+]+),"),i)[0].split()[0],re.findall(re.compile(r"c12=\s([0-9\.\-e\+]+)"),i)[0].split()[0]])
functype=pd.DataFrame(tmp,columns=["C6","C12"])

In [78]:
tmp2=[]
for i in range(int(math.sqrt(atnr))):
    tmp1=[]
    for j in range(int(math.sqrt(atnr))):
        tmp1.append(re.findall(re.compile(r"c6=\s([0-9\.\-e\+]+),"),func[i*int(math.sqrt(atnr))+j])[0].split()[0])
    tmp2.append(tmp1)
C6=np.array(tmp2).astype(float)

In [79]:
tmp1=[]
for i in range(int(math.sqrt(atnr))):
    tmp2=[]
    for j in range(int(math.sqrt(atnr))):
        tmp2.append(re.findall(re.compile(r"c12=\s([0-9\.\-e\+]+)"),func[i*int(math.sqrt(atnr))+j])[0].split()[0])
    tmp1.append(tmp2)
C12=np.array(tmp1).astype(float)

In [80]:
# 获取半径参数，从C12和C6计算
# （C12/C6)^(1./6）
np.seterr(divide='ignore',invalid='ignore')
rad=np.empty(C6.shape[0])
for i in range(C6.shape[0]):
    rad[i]=0.5*10*pow((C12[i][i]/C6[i][i]),1/6)
# 如果NaN 则默认复制为氢的半径参数
rad[np.where(np.isnan(rad))]=1.2
total.insert(loc=14,column="radius",value=1.2)
# 把参数信息整合到total
for i in range(rad.shape[0]):
    total.loc[total["functype"]==i,"radius"]=rad[i]

In [81]:
# radBondi["C" ]= 1.7  ; radBondi["H" ]= 1.2
# radBondi["N" ]= 1.55 ; radBondi["HC"]= 1.3
# radBondi["O" ]= 1.5  ; radBondi["HN"]= 1.3
# radBondi["F" ]= 1.5  ; radBondi["HP"]= 1.3
# radBondi["SI"]= 2.1  ; radBondi["HO"]= 0.8
# radBondi["P" ]= 1.85 ; radBondi["HS"]= 0.8
# radBondi["S" ]= 1.8  # radBondi["BR"]= 1.85
# radBondi["CL"]= 1.7  # radBondi["I" ]= 1.98
total.loc[total['atomtype'].str.upper().str[0]=='C','radius']=1.7
total.loc[total['atomtype'].str.upper().str[0]=='N','radius']=1.55
total.loc[total['atomtype'].str.upper().str[0]=='O','radius']=1.5
total.loc[total['atomtype'].str.upper().str[0]=='F','radius']=1.5
total.loc[total['atomtype'].str.upper().str[:2]=='SI','radius']=2.1
total.loc[total['atomtype'].str.upper().str[0]=='P','radius']=1.85
total.loc[total['atomtype'].str.upper().str[0]=='S','radius']=1.8
total.loc[total['atomtype'].str.upper().str[:2]=='CL','radius']=1.7
total.loc[total['atomtype'].str.upper().str[0]=='H','radius']=1.2
total.loc[total['atomtype'].str.upper().str[:2]=='HC','radius']=1.3
total.loc[total['atomtype'].str.upper().str[:2]=='HN','radius']=1.3
total.loc[total['atomtype'].str.upper().str[:2]=='HP','radius']=0.8
total.loc[total['atomtype'].str.upper().str[:2]=='HS','radius']=0.8
total.loc[total['atomtype'].str.upper().str[:2]=='BR','radius']=1.85
total.loc[total['atomtype'].str.upper().str[0]=='I','radius']=1.95


In [82]:
def apbsread(f,pron,lign,comn):
    readcont="""read
    mol pqr  {comn}
    mol pqr  {pron}
    mol pqr  {lign}
end
    """.format(comn=comn,pron=pron,lign=lign)
    f.write(readcont)

In [83]:
def apbselec(f,fn,MGset,i,nx,ny,nz,cx,cy,cz,fx,fy,fz,cntx,cnty,cntz,temp,pdie,sdie,pbm,bcfl,srfm,chgm,swin,srad,sdens,pcharge,pconc,pradius,ncharge,nconc,nradius,calcforce,calcenergy):
    eleccont="""
ELEC name {fn}
    {MGset}
    mol {i}
    dime {nx} {ny} {nz}  # 格点数目
    cglen {cx} {cy} {cz}  # 粗略格点长度
    fglen {fx} {fy} {fz}  # 细密格点长度
    fgcent {cntx} {cnty} {cntz}  # 细密格点中心
    cgcent {cntx} {cnty} {cntz}  # 粗略格点中心
    
    temp {temp}  # 温度
    pdie {pdie}  # 溶质介电常数
    sdie {sdie}  # 溶剂介电常数, 真空1, 水78.54

    {pbm}        # PB方程求解方法, lpbe(线性), npbe(非线性), smbpe(大小修正)
    bcfl  {bcfl}         # 粗略格点PB方程的边界条件, zero, sdh/mdh(single/multiple Debye-Huckel), focus, map
    srfm  {srfm}       # 构建介质和离子边界的模型, mol(分子表面), smol(平滑分子表面), spl2/4(三次样条/7阶多项式)
    chgm  {chgm}        # 电荷映射到格点的方法, spl0/2/4, 三线性插值, 立方/四次B样条离散
    swin  {swin}       # 立方样条的窗口值, 仅用于 srfm=spl2/4

    srad  {srad}         # 溶剂探测半径
    sdens {sdens}          # 表面密度, 每A^2的格点数, (srad=0)或(srfm=spl2/4)时不使用

    ion charge {pcharge} conc {pconc} radius {pradius}  # 阳离子的电荷, 浓度, 半径
    ion charge {ncharge} conc {nconc} radius {nradius}  # 阴离子

    calcforce  {calcforce}
    calcenergy {calcenergy}
end
    """.format(fn=fn,MGset=MGset,i=i,
               nx=nx,ny=ny,nz=nz,
               cx=cx,cy=cy,cz=cz,
               fx=fx,fy=fy,fz=fz,
               cntx=cntx,cnty=cnty,cntz=cntz,
               temp=temp,pdie=pdie,sdie=sdie,
               pbm=pbm,bcfl=bcfl,srfm=srfm,chgm=chgm,swin=swin,srad=srad,sdens=sdens,
               pcharge=pcharge,pconc=pconc,pradius=pradius,
               ncharge=ncharge,nconc=nconc,nradius=nradius,
               calcforce=calcforce,calcenergy=calcenergy)
    f.write(eleccont)


In [84]:
def apbsapol(f,fn,i,atemp,asrfm,aswin,asrad,agamma,apress,abconc,asdens,adpos,agridx,agridy,agridz,calcforce,calcenergy):
    apolarcont="""
APOLAR name {fn}
    mol {i}

    temp  {atemp} # 温度
    srfm  {asrfm}   # 构建溶剂相关表面或体积的模型
    swin  {aswin}   # 立方样条窗口(A), 用于定义样条表面

    # SASA
    srad  {asrad}   # 探测半径(A)
    gamma {agamma}      # 表面张力(kJ/mol-A^2)

    #gamma const 0.0226778 3.84928
    #gamma const 0.027     0
    #gamma const 0.0301248 0         # AMBER-PB4 .0072*cal2J 表面张力, 常数

    press  {apress}     # 压力(kJ/mol-A^3)
    bconc  {abconc}    # 溶剂本体密度(A^3)
    sdens  {asdens}
    dpos  {adpos}
    grid   {agridx} {agridy} {agridz}

    calcforce  {calcforce}
    calcenergy {calcenergy}
end
    """.format(fn=fn,i=i,atemp=atemp,asrfm=asrfm,aswin=aswin,asrad=asrad,agamma=agamma,apress=apress,abconc=abconc,asdens=asdens,adpos=adpos,
               agridx=agridx,agridy=agridy,agridz=agridz,calcforce=calcforce,calcenergy=calcenergy)
    f.write(apolarcont)

In [95]:
traj=mda.Universe(xtcpath)
fidx=np.array(total["functype"])
qcharge=np.array(total["charge"])
np.set_printoptions(precision=4)

resPBSAcomps=total.loc[:,["index","resid","resname"]]
totalresPBSA=pd.DataFrame()
totalPBSA=pd.DataFrame()

for i in traj.trajectory:
    if i.time%dt==0:
        # test MM
        ag=traj.atoms
        xyz=ag.positions
        Atomcoord=xyz*0.1
        # 输出pro pqr文件
        pron='pro_'+str(traj.trajectory.time/1000)+'ns.pqr'
        fp=open(pron,"w")
        for index,row in total.loc[prolist].iterrows():
            fp.write(("%-6s%5s%s%4s%s%3s%s%s%4s%s%3s%8.3f%8.3f%8.3f%8.4f%8.4f \n")%("ATOM  ",row["index"]," ",row["atomname"]," ",row["resname"]," "," ",row["resid"]," ","   ",xyz[index][0],xyz[index][1],xyz[index][2],row["charge"],row["radius"]))
        fp.write("TER \nEND")
        fp.close()

        # 输出lig pqr文件
        lign='lig_'+str(traj.trajectory.time/1000)+'ns.pqr'
        fp=open(lign,"w")
        for index,row in total.loc[liglist].iterrows():
            fp.write(("%-6s%5s%s%4s%s%3s%s%s%4s%s%3s%8.3f%8.3f%8.3f%8.4f%8.4f \n")%("ATOM  ",row["index"]," ",row["atomname"]," ",row["resname"]," "," ",row["resid"]," ","   ",xyz[index][0],xyz[index][1],xyz[index][2],row["charge"],row["radius"]))
        fp.write("TER \nEND")
        fp.close()

        # 输出com pqr文件
        comn='com_'+str(traj.trajectory.time/1000)+'ns.pqr'
        fp=open(comn,"w")
        for index,row in total.loc[comlist].iterrows():
            fp.write(("%-6s%5s%s%4s%s%3s%s%s%4s%s%3s%8.3f%8.3f%8.3f%8.4f%8.4f \n")%("ATOM  ",row["index"]," ",row["atomname"]," ",row["resname"]," "," ",row["resid"]," ","   ",xyz[index][0],xyz[index][1],xyz[index][2],row["charge"],row["radius"]))
        fp.write("TER \nEND")
        fp.close()


        TmpArr=np.tile(Atomcoord,Atomcoord.shape[0])-Atomcoord.flatten()
        TmpArr=TmpArr.reshape(Atomcoord.shape[0],Atomcoord.shape[0],3)
        D=np.linalg.norm(TmpArr,axis=2)

        # vdw
        vdw=[]
        for i in prolist:
            tmp=0
            for j in liglist:
                tmp+=(C12[fidx[i]][fidx[j]]/(math.pow(D[i,j],12)))-(C6[fidx[i]][fidx[j]]/(math.pow(D[i,j],6)))
            vdw.append(tmp/2)
        tvdw=np.sum(vdw)

        #  ele
        ele=[]
        diele=2
        for i in prolist:
            tmp=0
            for j in liglist:
                tmp+=qcharge[i]*qcharge[j]*138.935/(D[i,j]*diele)
            ele.append(tmp/2)
        tele=np.sum(ele)

        prolistrick=np.insert(prolist,-1,liglist[0])
        prolistrick.sort()
        resInfo=total.loc[prolistrick,['resid','resname']]
        vdw.append(tvdw)
        ele.append(tele)
        resInfo['vdw']=vdw
        resInfo['cou']=ele
        resInfo=resInfo.groupby('resid',sort=False).sum(numeric_only=True)
        resInfo['MM']=resInfo["vdw"]+resInfo['cou']
        # print("*********************************************")
        # print(resInfo)
        # print("*********************************************")

        minXpro=(xyz[prolist,0]-total.loc[prolist]["radius"]).min()
        minYpro=(xyz[prolist,1]-total.loc[prolist]["radius"]).min()
        minZpro=(xyz[prolist,2]-total.loc[prolist]["radius"]).min()
        maxXpro=(xyz[prolist,0]+total.loc[prolist]["radius"]).max()
        maxYpro=(xyz[prolist,1]+total.loc[prolist]["radius"]).max()
        maxZpro=(xyz[prolist,2]+total.loc[prolist]["radius"]).max()
        minXlig=(xyz[liglist,0]-total.loc[liglist]["radius"]).min()
        minYlig=(xyz[liglist,1]-total.loc[liglist]["radius"]).min()
        minZlig=(xyz[liglist,2]-total.loc[liglist]["radius"]).min()
        maxXlig=(xyz[liglist,0]+total.loc[liglist]["radius"]).max()
        maxYlig=(xyz[liglist,1]+total.loc[liglist]["radius"]).max()
        maxZlig=(xyz[liglist,2]+total.loc[liglist]["radius"]).max()
        minXcom=min(minXpro,minXlig)
        minYcom=min(minYpro,minYlig)
        minZcom=min(minZpro,minZlig)
        maxXcom=max(maxXpro,maxXlig)
        maxYcom=max(maxYpro,maxYlig)
        maxZcom=max(maxZpro,maxZlig)
        minX=minXcom
        minY=minYcom
        minZ=minZcom
        maxX=maxXcom
        maxY=maxYcom
        maxZ=maxZcom
        lenX=max(maxX-minX,0.1)
        lenY=max(maxY-minY,0.1)
        lenZ=max(maxZ-minZ,0.1)
        cntx=round((maxX+minX)/2,4)
        cnty=round((maxY+minY)/2,4)
        cntz=round((maxZ+minZ)/2,4)
        cx=round(lenX*cfac,4)
        cy=round(lenY*cfac,4)
        cz=round(lenZ*cfac,4)
        fx=round(min(cx,lenX+fadd),4)
        fy=round(min(cy,lenY+fadd),4)
        fz=round(min(cz,lenZ+fadd),4)
        levN=4 # 划分级别
        t=pow(2,levN+1)
        nx=round(fx/df)-1
        ny=round(fy/df)-1
        nz=round(fz/df)-1
        nx=max(t*round(nx/t)+1,33)
        ny=max(t*round(ny/t)+1,33)
        nz=max(t*round(nz/t)+1,33)

        if gridType==0: # GMXPBSA
            fpre=1
            cfac=1.7
            fx=lenX+2*fadd
            fy=lenY+2*fadd
            fz=lenZ+2*fadd
            cx=fx*cfac
            cy=fy*cfac
            cz=fz*cfac
            nx=t*(int(fx/(t*df))+1+fpre)+1
            ny=t*(int(fy/(t*df))+1+fpre)+1
            nz=t*(int(fz/(t*df))+1+fpre)+1

        mem=200*nx*ny*nz/1024/1024

        ft=str(traj.trajectory.time/1000)
        apbsfn=str(ft)+".apbs"
        apbs=open(apbsfn,"w")
        apbsread(apbs,pron=pron,lign=lign,comn=comn)
        eleccom=ft+"ns_com"
        apbselec(apbs,eleccom,MGset,1,nx,ny,nz,cx,cy,cz,fx,fy,fz,cntx,cnty,cntz,temp,pdie,sdie,pbm,bcfl,srfm,chgm,swin,srad,sdens,pcharge,pconc,pradius,ncharge,nconc,nradius,"no","comps")
        eleccomvac=ft+"ns_com_VAC"
        apbselec(apbs,eleccomvac,MGset,1,nx,ny,nz,cx,cy,cz,fx,fy,fz,cntx,cnty,cntz,temp,pdie,1,pbm,bcfl,srfm,chgm,swin,srad,sdens,pcharge,pconc,pradius,ncharge,nconc,nradius,"no","comps")
        apolcom=ft+"ns_com_SAS"
        apbsapol(apbs,apolcom,1,atemp,asrfm,aswin,asrad,agamma,apress,abconc,asdens,adpos,agridx,agridy,agridz,"no","total")
        pe="""
print elecEnergy {eleccom} - {eleccomvac} end
print apolEnergy {apolcom} end
        """.format(eleccom=eleccom,eleccomvac=eleccomvac,apolcom=apolcom)
        apbs.write(pe)
        
        elecpro=ft+"ns_pro"
        apbselec(apbs,elecpro,MGset,2,nx,ny,nz,cx,cy,cz,fx,fy,fz,cntx,cnty,cntz,temp,pdie,sdie,pbm,bcfl,srfm,chgm,swin,srad,sdens,pcharge,pconc,pradius,ncharge,nconc,nradius,"no","comps")
        elecprovac=ft+"ns_pro_VAC"
        apbselec(apbs,elecprovac,MGset,2,nx,ny,nz,cx,cy,cz,fx,fy,fz,cntx,cnty,cntz,temp,pdie,1,pbm,bcfl,srfm,chgm,swin,srad,sdens,pcharge,pconc,pradius,ncharge,nconc,nradius,"no","comps")
        apolpro=ft+"ns_pro_SAS"
        apbsapol(apbs,apolpro,2,atemp,asrfm,aswin,asrad,agamma,apress,abconc,asdens,adpos,agridx,agridy,agridz,"no","total")
        pe="""
print elecEnergy {elecpro} - {elecprovac} end
print apolEnergy {apolpro} end
        """.format(elecpro=elecpro,elecprovac=elecprovac,apolpro=apolpro)
        apbs.write(pe)

        eleclig=ft+"ns_lig"
        apbselec(apbs,eleclig,MGset,3,nx,ny,nz,cx,cy,cz,fx,fy,fz,cntx,cnty,cntz,temp,pdie,sdie,pbm,bcfl,srfm,chgm,swin,srad,sdens,pcharge,pconc,pradius,ncharge,nconc,nradius,"no","comps")
        elecligvac=ft+"ns_lig_VAC"
        apbselec(apbs,elecligvac,MGset,3,nx,ny,nz,cx,cy,cz,fx,fy,fz,cntx,cnty,cntz,temp,pdie,1,pbm,bcfl,srfm,chgm,swin,srad,sdens,pcharge,pconc,pradius,ncharge,nconc,nradius,"no","comps")
        apollig=ft+"ns_lig_SAS"
        apbsapol(apbs,apollig,3,atemp,asrfm,aswin,asrad,agamma,apress,abconc,asdens,adpos,agridx,agridy,agridz,"no","total")
        pe="""
print elecEnergy {eleclig} - {elecligvac} end
print apolEnergy {apollig} end
        """.format(eleclig=eleclig,elecligvac=elecligvac,apollig=apollig)
        apbs.write(pe)
        apbs.close()

        # apbs run
        os.system("apbs {} > apbs_{}.out ".format(apbsfn,ft))

        with open("apbs_{}.out".format(ft)) as fapbs:
            lines=fapbs.readlines()

        AtomPB=re.compile(r"\s+Atom\s[0-9]+:\s\s([\-\.\+E0-9]+)\s+kJ/mol")
        AtomSA=re.compile(r"\s+SASA for atom\s[0-9]+:\s([\-\.\+E0-9]+)")
        Atoms=[]
        for line in lines:
            PBtmp=re.findall(AtomPB,line)
            if PBtmp!=[]:
                Atoms.append(PBtmp)
            SAtmp=re.findall(AtomSA,line)
            if SAtmp!=[]:
                Atoms.append(SAtmp)

        eleccom1=Atoms[len(comlist)]                        # Multiple Debye-Huckel sphere boundary conditions
        eleccom2=Atoms[len(comlist):(len(comlist)*2)]                # Boundary conditions from focusing
        eleccomvac1=Atoms[(len(comlist)*2):(len(comlist)*3)]
        eleccomvac2=Atoms[(len(comlist)*3):(len(comlist)*4)]
        elecpro1=Atoms[(len(comlist)*4):(len(comlist)*4+len(prolist))]
        elecpro2=Atoms[(len(comlist)*4+len(prolist)):(len(comlist)*4+len(prolist)*2)]
        elecprovac1=Atoms[(len(comlist)*4+len(prolist)*2):(len(comlist)*4+len(prolist)*3)]
        elecprovac2=Atoms[(len(comlist)*4+len(prolist)*3):(len(comlist)*4+len(prolist)*4)]
        eleclig1=Atoms[(len(comlist)*4+len(prolist)*4):(len(comlist)*4+len(prolist)*4+(len(liglist)))]
        eleclig2=Atoms[(len(comlist)*4+len(prolist)*4+(len(liglist))):(len(comlist)*4+len(prolist)*4+(len(liglist))*2)]
        elecligvac1=Atoms[(len(comlist)*4+len(prolist)*4+(len(liglist))*2):(len(comlist)*4+len(prolist)*4+(len(liglist))*3)]
        elecligvac2=Atoms[(len(comlist)*4+len(prolist)*4+(len(liglist))*3):(len(comlist)*4+len(prolist)*4+(len(liglist))*4)]
        apolcomsas=gamma*np.array(Atoms[(len(comlist)*4+len(prolist)*4+(len(liglist))*4):(len(comlist)*5+len(prolist)*4+(len(liglist))*4)]).astype("float")+offset
        apolprosas=gamma*np.array(Atoms[(len(comlist)*5+len(prolist)*4+(len(liglist))*4):(len(comlist)*5+len(prolist)*5+(len(liglist))*4)]).astype("float")+offset
        apolligsas=gamma*np.array(Atoms[(len(comlist)*5+len(prolist)*5+(len(liglist))*4):(len(comlist)*5+len(prolist)*5+(len(liglist))*5)]).astype("float")+offset

        pb=[]
        sa=[]
        totene=[]
        for line in lines:
            pol=re.findall(re.compile(r"Global net ELEC energy = ([\-\.\+E0-9]+)"),line)
            apol=re.findall(re.compile(r"Global net APOL energy = ([\-\.\+E0-9]+)"),line)
            if pol!=[]:
                pb.append(pol)
            if apol!=[]:
                sa.append(apol)
        totene.append([pb,sa])
        dmmpbsa=np.array(totene).flatten().astype("float")
        dmmpbsa[3]=gamma*dmmpbsa[3]+offset
        dmmpbsa[4]=gamma*dmmpbsa[4]+offset
        dmmpbsa[5]=gamma*dmmpbsa[5]+offset
        dPB=dmmpbsa[0]-dmmpbsa[1]-dmmpbsa[2]
        dSA=dmmpbsa[3]-dmmpbsa[4]-dmmpbsa[5]
        dMMPBSA=resInfo["MM"].sum()+dPB+dSA
        dmmpbsa=np.append(dmmpbsa,[resInfo["vdw"].sum(),resInfo["cou"].sum(),resInfo["MM"].sum(),dPB,dSA,dMMPBSA])
        totalPBSA=pd.concat([totalPBSA,pd.DataFrame(np.insert(dmmpbsa,0,ft).reshape(1,13),columns=["Time(ns)","PB_com","PB_pro","PB_lig","SA_com","SA_pro","SA_lig","vdw","cou","MM","dPB","dSA","MM-PBSA"],index=["1"])])
        # print("*********************************************")
        # print(totalPBSA)
        # print("*********************************************")        
        ResComPBSA=pd.DataFrame([np.array(eleccom1).flatten(), np.array(eleccom2).flatten(),np.array(eleccomvac1).flatten(),np.array(eleccomvac2).flatten(),np.array(apolcomsas).flatten()]).T.astype("float")
        ResComPBSA.columns=["Elec_DH","Elec_BCF","ElecVac_DH","ElecVac_BCF","SA"]
        ResLigPBSA=pd.DataFrame([np.array(eleclig1).flatten(), np.array(eleclig2).flatten(),np.array(elecligvac1).flatten(),np.array(elecligvac2).flatten(),np.array(apolligsas).flatten()]).T.astype("float")
        ResLigPBSA.columns=["Elec_DH","Elec_BCF","ElecVac_DH","ElecVac_BCF","SA"]
        ResProPBSA=pd.DataFrame([np.array(elecpro1).flatten(), np.array(elecpro2).flatten(),np.array(elecprovac1).flatten(),np.array(elecprovac2).flatten(),np.array(apolprosas).flatten()]).T.astype("float")
        ResProPBSA.columns=["Elec_DH","Elec_BCF","ElecVac_DH","ElecVac_BCF","SA"]

        tmp=ResComPBSA-pd.concat([ResProPBSA,ResLigPBSA],ignore_index=True)
        resPBSAcomps["PB"]=tmp["Elec_BCF"]-tmp["ElecVac_BCF"]
        resPBSAcomps["SA"]=tmp["SA"]

        resPBSA=resPBSAcomps.drop_duplicates("resid",ignore_index=True).loc[:,["resid","resname"]]
        resPBSA.set_index("resid")
        resPBSA["vdw"]=np.array(resInfo["vdw"])
        resPBSA["cou"]=np.array(resInfo["cou"])
        resPBSA["MM"]=np.array(resInfo["MM"])
        resPBSA["PB"]=np.array(resPBSAcomps.groupby("resid").sum({"PB":"sum","SA":"sum"})["PB"])
        resPBSA["SA"]=np.array(resPBSAcomps.groupby("resid").sum({"PB":"sum","SA":"sum"})["SA"])
        resPBSA.columns=[[ft+"ns",ft+"ns",ft+"ns",ft+"ns",ft+"ns",ft+"ns",ft+"ns"],["resid","resname","vdw","cou","MM","PB","SA"]]
        # print("*********************************************")
        # print(resPBSA)
        # print("*********************************************")
        totalresPBSA=pd.concat([totalresPBSA,resPBSA],axis=1)
        print("{}ns is done".format(ft))


# 输出不同时间残基能量贡献
totalPBSA=pd.concat([totalPBSA,pd.DataFrame(totalPBSA.mean()).T],axis=0)
totalPBSA.iloc[-1,0]="Mean"
totalresPBSA.to_csv("TotalresDecomps.csv",index=False)
# 输出平均残基能量贡献
avgresPBSA=pd.DataFrame()
totalresPBSA.columns=totalresPBSA.columns.droplevel()
if totalPBSA.shape[0]==2:
    avgresPBSA["resid"]=totalresPBSA["resid"].iloc[:,0]
    avgresPBSA["resname"]=totalresPBSA["resname"].iloc[:,0]
    avgresPBSA["vdw"]=totalresPBSA["vdw"]
    avgresPBSA["cou"]=totalresPBSA["vdw"]
    avgresPBSA["MM"]=totalresPBSA["MM"]
    avgresPBSA["PB"]=totalresPBSA["PB"]
    avgresPBSA["SA"]=totalresPBSA["SA"]
else: 
    avgresPBSA["resid"]=totalresPBSA["resid"].iloc[:,0]
    avgresPBSA["resname"]=totalresPBSA["resname"].iloc[:,0]
    avgresPBSA["vdw"]=totalresPBSA["vdw"].mean(axis=1)
    avgresPBSA["cou"]=totalresPBSA["vdw"].mean(axis=1)
    avgresPBSA["MM"]=totalresPBSA["MM"].mean(axis=1)
    avgresPBSA["PB"]=totalresPBSA["PB"].mean(axis=1)
    avgresPBSA["SA"]=totalresPBSA["SA"].mean(axis=1)
# 输出总的MMPBSA
totalPBSA.to_csv("TotalMMPBSA.csv",index=False)

# 输出平均残基能量贡献
avgresPBSA.to_csv("avgresMMPSA.csv",index=False)

# 删除文件
os.system("rm *.apbs")
os.system("rm *.out")
os.system("rm *.pqr")

0.0ns is done
10.0ns is done


0

In [94]:
totalresPBSA["resid"].iloc[:,0]

0        1
1        2
2        3
3        4
4        5
      ... 
194    195
195    196
196    197
197    198
198    199
Name: resid, Length: 199, dtype: int64

---

Total electrostatic energy = Fixed charge energy + Mobile charge energy + Dielectric energy   
Fixed charge energy= the sum of per-atom energies    
Elec_DH    Multiple Debye-Huckel sphere boundary conditions    
Elec_BCF   Boundary conditions from focusing  使用该能量！  